# Gibbs Sampling

This is useful for sampling from complex distribution $-$ multi-dimensional distribution  by component-wise sampling indenpendently. For example, we want to model two parameters, $\theta_{1}$, $\theta_{2}$ and $\theta_{3}$.

1. Initialize $\theta_{1}^{0}$, $\theta_{2}^{0}$ and $\theta_{3}^{0}$ as some value
2. for each iteration $i$:
 - crate a new value $\theta_{1}^{i}$ conditioned on values $\theta_{2}^{i-1}$ and $\theta_{3}^{i-1}$
 - crate a new value $\theta_{2}^{i}$ conditioned on values $\theta_{1}^{i}$ and $\theta_{3}^{i-1}$
 - crate a new value $\theta_{3}^{i}$ conditioned on values $\theta_{1}^{i}$ and $\theta_{2}^{i}$

![example](https://github.com/Donghwa-KIM/Donghwa-KIM.github.io/blob/master/picture/example_sequence.jpg)